# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo ```SINASC_RO_2019.csv``` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo ```estrutura_sinasc_para_CD.pdf``` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
``` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
```
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis ```['ESTCIVMAE', 'CONSULTAS']``` possuem o código ```9```, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código ```9```.<br>
6. Substitua os valores faltantes da quantitativa (```QTDFILVIVO```) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção ```inplace = True```.

In [1]:
import pandas as pd
import requests

# 1) Há linhas duplicadas?
sinasc = pd.read_csv('SINASC_RO_2019.csv')
registros,_ = sinasc.shape
print('número de registros: ',registros)
registros_nao_dupl,_ = sinasc.drop_duplicates().shape
# Não há duplicados
print('número de registros não duplicados',registros_nao_dupl)

número de registros:  27028
número de registros não duplicados 27028


In [2]:
# 2) Conte o número de valores missing por variável
pd.DataFrame(sinasc.isna().sum(), columns=['Total de valores missing']).transpose()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
Total de valores missing,0,115,0,0,0,317,312,2907,1573,2098,...,0,0,0,0,0,0,1,1,1,1


In [3]:
# 3) crie uma seleção dessa base somente com as colunas que interessam
# ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO','GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
lista_colunas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO','GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
pd.DataFrame(sinasc[lista_colunas].isna().sum(), columns=['Total de valores missing']).transpose()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
Total de valores missing,0,0,317,312,1573,1232,79,0,103


In [4]:
# 4) remova todos os registros com Apgar5 não preenchido
sinasc.dropna(subset = 'APGAR5', inplace = True)
# conte novamente o número de linhas e o número de missings
sinasc_na = pd.DataFrame(sinasc[lista_colunas].isna().sum(), columns=['Total de valores missing']).transpose()
registros,_ = sinasc.shape
print('número de registros: ',registros)
sinasc_na

número de registros:  26925


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
Total de valores missing,0,0,315,310,1566,1216,76,0,0


In [5]:
# 5) Vamos assumir que o não preenchido é o mesmo que o código 9
# as variáveis ['ESTCIVMAE', 'CONSULTAS']
# após a exclusão dos itens missing da coluna 'APGAR5', a coluna 'CONSULTAS' não apresenta valores missing.
sinasc.loc[sinasc['ESTCIVMAE'].isna(), 'ESTCIVMAE'] = 9
sinasc_na = pd.DataFrame(sinasc[lista_colunas].isna().sum(), columns=['Total de valores missing']).transpose()
sinasc_na

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
Total de valores missing,0,0,0,310,1566,1216,76,0,0


In [6]:
# 6) Substitua os valores faltantes da quantitativa (QTDFILVIVO) por zero.
sinasc.loc[sinasc['QTDFILVIVO'].isna(), 'QTDFILVIVO'] = 0
sinasc_na = pd.DataFrame(sinasc[lista_colunas].isna().sum(), columns=['Total de valores missing']).transpose()
sinasc_na

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
Total de valores missing,0,0,0,310,0,1216,76,0,0


In [7]:
# 7) Das restantes, decida que valor te parece mais adequado, preencha e justifique
percentage = (sinasc[lista_colunas].isna().sum() / len(sinasc)) * 100
sinasc_per = pd.DataFrame(percentage[['ESCMAE', 'GESTACAO', 'GRAVIDEZ']], columns=['Porcentagem de missing']).transpose()
sinasc_per

,ESCMAE,GESTACAO,GRAVIDEZ
Porcentagem de missing,1.151346,4.516249,0.282266


In [8]:
# A configuração method="bfill" preenche todos os valores NaN de DataFrame com o valor após o valor NaN na mesma coluna
sinasc['ESCMAE'].fillna(method="bfill", inplace = True)
sinasc['GESTACAO'].fillna(method="bfill", inplace = True)
sinasc['GRAVIDEZ'].fillna(method="bfill", inplace = True)
sinasc_na = pd.DataFrame(sinasc[lista_colunas].isna().sum(), columns=['Total de valores missing']).transpose()
sinasc_na

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
Total de valores missing,0,0,0,0,0,0,0,0,0


**Justificativa**: a coluna **'GESTACAO'** é a única que apresenta maior representatividade, com cerca de 4,52%. As demais pela pouca representatividade poderiam até serem excluídas. Contudo, optou-se por substituir os valores **'missing'** de todas as 03 (três) colunas pelo valor anterior, fazendo com que, no final, a participação de cada categoria fique próxima daquela verificada nos eventos com registros **não missing**.

In [9]:
# 8) Crie uma categorização da variável APGAR5 e calcule as frequências dessa categorização
sinasc.loc[sinasc['APGAR5'] <= 3, 'APGAR5_CAT'] = 'asfixia severa'
sinasc.loc[(sinasc['APGAR5'] > 3) & (sinasc['APGAR5'] <= 5),
           'APGAR5_CAT'] = 'asfixia moderada'
sinasc.loc[(sinasc['APGAR5'] > 5) & (sinasc['APGAR5'] <= 7),
           'APGAR5_CAT'] = 'asfixia leve'
sinasc.loc[(sinasc['APGAR5'] > 7), 'APGAR5_CAT'] = 'normal'
sinasc.APGAR5_CAT.value_counts()

normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: APGAR5_CAT, dtype: int64

In [10]:
dummies = pd.get_dummies(sinasc['APGAR5_CAT'])
sinasc = pd.concat([sinasc, dummies], axis=1)
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,munResUf,munResLat,munResLon,munResAlt,munResArea,APGAR5_CAT,asfixia leve,asfixia moderada,asfixia severa,normal
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal,0,0,0,1
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal,0,0,0,1
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal,0,0,0,1
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,Rondônia,-12.13178,-61.85308,397.0,3958.273,normal,0,0,0,1
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal,0,0,0,1


In [11]:
print('Frequência de asfixia leve: ', sinasc['asfixia leve'].sum())
print('Frequência de asfixia moderada: ', sinasc['asfixia moderada'].sum())
print('Frequência de asfixia severa: ', sinasc['asfixia severa'].sum())
print('Frequência de normal: ', sinasc['normal'].sum())

Frequência de asfixia leve:  320
Frequência de asfixia moderada:  68
Frequência de asfixia severa:  74
Frequência de normal:  26463


In [12]:
# 9) Renomeie as variáveis para que fiquem no snake case
sinasc.rename(columns = {'asfixia leve':'ASFIXIA_LEVE','asfixia moderada':'ASFIXIA_MODERADA',
                        'asfixia severa':'ASFIXIA_SEVERA','normal':'NORMAL'}, inplace = True)
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,munResUf,munResLat,munResLon,munResAlt,munResArea,APGAR5_CAT,ASFIXIA_LEVE,ASFIXIA_MODERADA,ASFIXIA_SEVERA,NORMAL
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal,0,0,0,1
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal,0,0,0,1
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal,0,0,0,1
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,Rondônia,-12.13178,-61.85308,397.0,3958.273,normal,0,0,0,1
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,Rondônia,-11.93554,-61.99982,338.0,7067.025,normal,0,0,0,1
